In [2]:
import awswrangler  as wr
import pandas as pd
# import pandas.io.json
import pymysql
import datetime
import json


# load sql
sql = '''
select *
from moyo_scraping.mno_scraping_result
where 1=1
'''
sql_v1 = ''' 
select *
from moyo_scraping.mno_scraping_result
where 1=1
    and version = 1 
'''
sql_v2 = '''
select *
from moyo_scraping.mno_scraping_result
where 1=1

    and version = 2
'''
df_v1= wr.athena.read_sql_query(sql=sql_v1, database="moyo_order")
df_v2= wr.athena.read_sql_query(sql=sql_v2, database="moyo_order")
df = wr.athena.read_sql_query(sql=sql, database="moyo_order")

request_result = zip(df_v1.request_id, df_v1.result)
parsed_data_v1 = []
for (request_id, result) in request_result:
    parsed_data_v1.append({'request_id': request_id, **json.loads(result)})

request_result = zip(df_v2.request_id, df_v2.result)
parsed_data_v2 = []
for (request_id, result) in request_result:
    parsed_data_v2.append({'request_id': request_id, **json.loads(result)})

request_result = zip(df.request_id, df.result)
parsed_data = []
for (request_id, result) in request_result:
    parsed_data.append({'request_id': request_id, **json.loads(result)})


# load json
# main 

In [3]:

df_scraping_main = pd.json_normalize (parsed_data 
                , sep = '_'
                )

In [35]:
df_scraping_main.head()

,request_id,paymentHistory,mobileUsageHistory,familyBundleMembers,profile_name,profile_telecom,profile_phoneNumber,contract_name,contract_endDate,contract_startDate,contract_hasExpired,contract_cancellationFee,mobilePlan_fee,mobilePlan_name,mobilePlan_totalDiscountRate,mobilePlan_totalDiscountAmount,contract,billingHistory
0,23,"[{'paidDate': '2023-04-21', 'billedDate': '202...","[{'callUsage': 107.0, 'dataUsage': 9.0, 'yearM...",[],유창*,KT,010-65**-*617,5G스폰서,2023-11-22,2021-11-23,False,82146.0,0,5G 슬림,0.0,0,NaN,NaN
1,25,"[{'paidDate': '2023-04-12', 'billedDate': '202...","[{'callUsage': 114.0, 'dataUsage': 11.3, 'year...",[],오진*,KT,010-23**-*491,5G스폰서,2024-06-08,2022-06-10,False,97242.0,0,5G 슬림_Y덤,0.0,0,NaN,NaN
2,26,"[{'paidDate': '2023-04-13', 'billedDate': '202...","[{'callUsage': 1036.0, 'dataUsage': 128.4, 'ye...",[],이*진,SKT,01066**56**,선택약정할인제도 (24 개월),2023-07-02,2021-07-02,False,46990.0,0,5GX 레귤러,0.0,0,NaN,NaN
3,30,"[{'paidDate': '2023-04-14', 'billedDate': '202...","[{'callUsage': 108.0, 'dataUsage': 30.5, 'year...",[],정*선,SKT,01074**13**,선택약정할인제도 (24 개월),2024-10-12,2022-10-13,False,128640.0,0,5GX 레귤러,0.0,0,NaN,NaN
4,31,"[{'paidDate': '2023-04-25', 'billedDate': '202...","[{'callUsage': 250.0, 'dataUsage': 9.5, 'yearM...",[],박*진,SKT,01020**84**,NaN,NaN,NaN,NaN,NaN,0,슬림,0.0,0,NaN,NaN


In [1]:
df_scraping_main = df_scraping_main.drop(columns=['paymentHistory', 'mobileUsageHistory', 'familyBundleMembers','billingHistory'])

NameError: name 'df_scraping_main' is not defined

In [39]:

# df_scraping_main = pd.json_normalize (parsed_data 
#                 , sep = '_'
#                 )

# remove column [paymenthistory, mobileusagehistory, familyboundlemembers]
# df_scraping_main = df_scraping_main.drop(columns=['paymenthistory', 'mobileusagehistory', 'familybundlemembers'])


bucket = 'moyo-mart'
table_name = 'scraping_main'
res = wr.s3.to_parquet(
    df=df_scraping_main,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

In [29]:
df_scraping_main.head()

,request_id,paymenthistory,mobileusagehistory,familybundlemembers,profile_name,profile_telecom,profile_phonenumber,contract_name,contract_enddate,contract_startdate,contract_hasexpired,contract_cancellationfee,mobileplan_fee,mobileplan_name,mobileplan_totaldiscountrate,mobileplan_totaldiscountamount,contract,billinghistory
0,23,"[{'paidDate': '2023-04-21', 'billedDate': '202...","[{'callUsage': 107.0, 'dataUsage': 9.0, 'yearM...",[],유창*,KT,010-65**-*617,5G스폰서,2023-11-22,2021-11-23,False,82146.0,0,5G 슬림,0.0,0,NaN,NaN
1,25,"[{'paidDate': '2023-04-12', 'billedDate': '202...","[{'callUsage': 114.0, 'dataUsage': 11.3, 'year...",[],오진*,KT,010-23**-*491,5G스폰서,2024-06-08,2022-06-10,False,97242.0,0,5G 슬림_Y덤,0.0,0,NaN,NaN
2,26,"[{'paidDate': '2023-04-13', 'billedDate': '202...","[{'callUsage': 1036.0, 'dataUsage': 128.4, 'ye...",[],이*진,SKT,01066**56**,선택약정할인제도 (24 개월),2023-07-02,2021-07-02,False,46990.0,0,5GX 레귤러,0.0,0,NaN,NaN
3,30,"[{'paidDate': '2023-04-14', 'billedDate': '202...","[{'callUsage': 108.0, 'dataUsage': 30.5, 'year...",[],정*선,SKT,01074**13**,선택약정할인제도 (24 개월),2024-10-12,2022-10-13,False,128640.0,0,5GX 레귤러,0.0,0,NaN,NaN
4,31,"[{'paidDate': '2023-04-25', 'billedDate': '202...","[{'callUsage': 250.0, 'dataUsage': 9.5, 'yearM...",[],박*진,SKT,01020**84**,<NA>,<NA>,<NA>,<NA>,NaN,0,슬림,0.0,0,NaN,NaN


In [30]:
df_scraping_main = df_scraping_main.drop(columns=['paymenthistory', 'mobileusagehistory', 'familybundlemembers'])

In [31]:
df_scraping_main.head()

,request_id,profile_name,profile_telecom,profile_phonenumber,contract_name,contract_enddate,contract_startdate,contract_hasexpired,contract_cancellationfee,mobileplan_fee,mobileplan_name,mobileplan_totaldiscountrate,mobileplan_totaldiscountamount,contract,billinghistory
0,23,유창*,KT,010-65**-*617,5G스폰서,2023-11-22,2021-11-23,False,82146.0,0,5G 슬림,0.0,0,NaN,NaN
1,25,오진*,KT,010-23**-*491,5G스폰서,2024-06-08,2022-06-10,False,97242.0,0,5G 슬림_Y덤,0.0,0,NaN,NaN
2,26,이*진,SKT,01066**56**,선택약정할인제도 (24 개월),2023-07-02,2021-07-02,False,46990.0,0,5GX 레귤러,0.0,0,NaN,NaN
3,30,정*선,SKT,01074**13**,선택약정할인제도 (24 개월),2024-10-12,2022-10-13,False,128640.0,0,5GX 레귤러,0.0,0,NaN,NaN
4,31,박*진,SKT,01020**84**,<NA>,<NA>,<NA>,<NA>,NaN,0,슬림,0.0,0,NaN,NaN


In [16]:
df_scraping_main = df_scraping_main.drop(columns=['paymenthistory', 'mobileusagehistory', 'familybundlemembers'])

In [17]:
df_scraping_main.head()

,request_id,contract,profile_name,profile_telecom,profile_phonenumber,mobileplan_fee,mobileplan_name,mobileplan_totaldiscountrate,mobileplan_totaldiscountamount,contract_name,contract_enddate,contract_startdate,contract_hasexpired,contract_cancellationfee,billinghistory
0,1,NaN,임*빈,SKT,01071**02**,0,0플랜 슈퍼히어로,0.0,0,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1,8,NaN,정*진,SKT,01056**02**,0,베이직,0.0,0,요금약정할인제도 (730 일),2023-05-29,2021-05-29,False,0.0,NaN
2,9,NaN,오*수,SKT,01093**49**,0,T플랜 에센스,0.0,0,선택약정할인제도 (12 개월),2023-05-19,2022-05-19,False,10570.0,NaN
3,12,NaN,신지*,KT,010-97**-*101,0,슈퍼플랜 베이직 초이스,0.0,0,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,14,NaN,임*지,SKT,01035**11**,0,5GX 프라임,0.0,0,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [42]:
df_scraping_main.columns

Index(['request_id', 'paymentHistory', 'mobileUsageHistory',
       'familyBundleMembers', 'profile_name', 'profile_telecom',
       'profile_phoneNumber', 'contract_name', 'contract_endDate',
       'contract_startDate', 'contract_hasExpired', 'contract_cancellationFee',
       'mobilePlan_fee', 'mobilePlan_name', 'mobilePlan_totalDiscountRate',
       'mobilePlan_totalDiscountAmount', 'contract', 'billingHistory'],
      dtype='object')

In [45]:
df_scraping_main = pd.json_normalize (parsed_data 
                , sep = '_'
                )

df_scraping_main = df_scraping_main.drop(columns=['paymentHistory', 'mobileUsageHistory', 'familyBundleMembers'],errors='ignore')
bucket = 'moyo-mart'
table_name = 'scraping_main'
res = wr.s3.to_parquet(
    df=df_scraping_main,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

In [4]:
	sql = '''
	select *
	from moyo_scraping.mno_scraping_result
	where 1=1
	'''
	sql_v1 = ''' 
	select *
	from moyo_scraping.mno_scraping_result
	where 1=1
	    and version = 1 
	'''
	sql_v2 = '''
	select *
	from moyo_scraping.mno_scraping_result
	where 1=1
	    and version = 2
	'''
	df_v1= wr.athena.read_sql_query(sql=sql_v1, database="moyo_order")
	df_v2= wr.athena.read_sql_query(sql=sql_v2, database="moyo_order")
	df = wr.athena.read_sql_query(sql=sql, database="moyo_order")
	
	request_result = zip(df_v1.request_id, df_v1.result)
	parsed_data_v1 = []
	for (request_id, result) in request_result:
	    parsed_data_v1.append({'request_id': request_id, **json.loads(result)})
	
	request_result = zip(df_v2.request_id, df_v2.result)
	parsed_data_v2 = []
	for (request_id, result) in request_result:
	    parsed_data_v2.append({'request_id': request_id, **json.loads(result)})
	
	request_result = zip(df.request_id, df.result)
	parsed_data = []
	for (request_id, result) in request_result:
	    parsed_data.append({'request_id': request_id, **json.loads(result)})
	
	
	# load json
	# main 
	df_scraping_main = pd.json_normalize (parsed_data 
	                , sep = '_'
	                )
	df_scraping_main = df_scraping_main.drop(columns=['paymentHistory', 'mobileUsageHistory', 'familyBundleMembers'],errors='ignore') #2023-06-13 수정
	
	bucket = 'moyo-mart'
	table_name = 'scraping_main'
	res = wr.s3.to_parquet(
	    df=df_scraping_main,
	    path=f"s3://{bucket}/{table_name}/",
	    dataset=True,
	    database="moyo_mart",
	    table=table_name,
	    mode="overwrite_partitions", 
	    # partition_cols=['date_ymd']
	)